L’objectif de cette section est d’évaluer la capacité d’un modèle de Machine Learning à extraire une propriété physique simple d’un état quantique à partir de mesures bruitées. Dans le cadre de la tomographie quantique, on souhaite en général reconstruire l’état réel du qubit, représenté par son vecteur de Bloch (X_real, Y_real, Z_real). Cependant, avant d’aborder cette tâche plus complexe de régression, il est utile de commencer par un problème plus simple : déterminer si l’état appartient à l’hémisphère nord ou à l’hémisphère sud de la sphère de Bloch.

Pour cela, on définit une étiquette binaire directement à partir de l’état réel simulé : label = 1 si Z_real ≥ 0, et 0 sinon. Cette propriété géométrique simple découle uniquement de l’état réel, c’est-à-dire l’état effectivement mesuré après bruit statistique et éventuellement bruit de canal. L’objectif de la classification est donc de prédire cette étiquette en utilisant uniquement les mesures brutes bruitées, à savoir les valeurs X_mean, Y_mean et Z_mean obtenues à partir d’un nombre limité de shots.

Ce test sert de validation préalable essentielle. S’il est impossible pour un modèle de classification de récupérer une propriété globale aussi simple que le signe de Z_real, il est illusoire d’espérer réussir une reconstruction complète du vecteur de Bloch réel via une régression plus fine. À l’inverse, si la classification réussit malgré le bruit statistique, cela montre que les features contiennent bien une partie exploitable de l’information physique.

En résumé, cette classification n’a pas pour but de résoudre la tomographie complète, mais de tester de manière contrôlée et progressive la capacité d’un modèle à extraire une information physique réelle à partir de mesures bruitées. Ce type d’analyse constitue une étape préparatoire logique avant de s’engager dans des tâches de reconstruction plus précises de l’état quantique.

In [21]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Imports Qiskit
from qiskit.circuit.library import ZZFeatureMap
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit.primitives import Sampler
from qiskit_machine_learning.algorithms import QSVC

from dataset_build.clf_dtSet_hugo import build_purity_classification_dataset

We can use the Strong law of numbers to prepare the n_shots value. Using graphical plots to see the convergence of the random average variable.

In [22]:
# Dataset creation

df_purity, X, y = build_purity_classification_dataset(
    n_shots=100,
    n_states_total=500,
    mixed_proportion=0.3  # proportion of mixed states
)

print(f"Dataset shape: {df_purity.shape}")
print(f"Labels distribution:\n{y.value_counts()}")
print(f"\nDataset columns:\n{df_purity.columns.tolist()}")
print(f"\X columns (à revoir):\n{X.columns.tolist()}")

Dataset shape: (500, 14)
Labels distribution:
label_purity
1    350
0    150
Name: count, dtype: int64

Dataset columns:
['X_mean', 'Y_mean', 'Z_mean', 'X_real', 'Y_real', 'Z_real', 'theta_ideal', 'phi_ideal', 'X_ideal', 'Y_ideal', 'Z_ideal', 'bloch_radius_real', 'is_pure', 'label_purity']
\X columns (à revoir):
['X_mean', 'Y_mean', 'Z_mean', 'bloch_radius_real']


In [ ]:
df_purity.head(10)

In [ ]:
df_purity.describe()

In [ ]:
df_purity.hist()

The X,Y and Z are following a unimodal Gaussian distribution, so that the dataset creation is normal.

In [ ]:
# correlation matrix
plt.figure(figsize=(8, 6))
sns.heatmap(df_purity.corr(), annot=True, cmap='coolwarm', fmt='.2f', cbar_kws={"shrink": .8})
plt.title('Features correlation matrix')
plt.tight_layout()
plt.show()

Putting bloch_radius_real creates an overfitting training, so we take it out from the dataset

In [ ]:
X_purity=df_purity[['X_mean','Y_mean','Z_mean']]
X_purity.head()

In [ ]:
y=df_purity['label_purity']

Transforming data for training models

In [ ]:
scaler=StandardScaler()


X_purity_scaled=scaler.fit_transform(X_purity)

X_train, X_test, y_train, y_test = train_test_split(
    X_purity_scaled, y, test_size=0.2, random_state=42, stratify=y)


print(f"Training set size: {X_train.shape[0]} samples")

In [23]:

# 2) Features: exemple simple (tu peux changer)
feature_cols = [
    "X_mean", "Y_mean", "Z_mean",
    "theta_ideal", "phi_ideal",
    "bloch_radius_real"
]
X = df_purity[feature_cols].values

# 3) Labels
y = df_purity["label_purity"].values  # ou df["is_pure"].astype(int).values

# 4) Train / test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

scaler_enc = MinMaxScaler(feature_range=(0, np.pi))

X_train_enc = scaler_enc.fit_transform(X_train)
X_test_enc  = scaler_enc.transform(X_test)

num_features = X_train_enc.shape[1]

feature_map = ZZFeatureMap(
    feature_dimension=num_features,
    reps=2,                 # profondeur entanglement, à tuner
    entanglement="full"     # ou "linear"
)

quantum_kernel = FidelityQuantumKernel(
    feature_map=feature_map
)



Training SVC models

In [24]:
qsvc = QSVC(quantum_kernel=quantum_kernel, C=1.0)

qsvc.fit(X_train_enc, y_train)
train_acc_qsvc = qsvc.score(X_train_enc, y_train)
test_acc_qsvc  = qsvc.score(X_test_enc, y_test)
print(f"QSVC - train acc: {train_acc_qsvc:.3f}, test acc: {test_acc_qsvc:.3f}")

QSVC - train acc: 0.995, test acc: 0.970


In [27]:
K_train = quantum_kernel.evaluate(X_train_enc, X_train_enc)
K_test  = quantum_kernel.evaluate(X_test_enc,  X_train_enc)

# 2) SVM classique sur noyau pré-calculé
svc_q = SVC(kernel="precomputed", C=1.0)
svc_q.fit(K_train, y_train)

# 3) Prédictions SANS recalcul de kernel
y_train_pred = svc_q.predict(K_train)
y_test_pred  = svc_q.predict(K_test)

print("\n=== Classification report (test) ===")
print(classification_report(y_test, y_test_pred))

KeyboardInterrupt: 

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': ['scale', 'auto']
}

svc = SVC()

# GridSearchCV
grid_search = GridSearchCV(
    estimator=svc,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    scoring='accuracy',
    verbose=1,
    n_jobs=-1  # utiliser tous les processeurs disponibles
)

# Entraîner le modèle avec recherche en grille
print("Démarrage de GridSearchCV...")
grid_search.fit(X_train, y_train)

print(f"\nMeilleurs paramètres : {grid_search.best_params_}")
print(f"Meilleur score CV : {grid_search.best_score_:.4f}")

In [ ]:

# Evaluating on the test set
y_pred = grid_search.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred)
print(f"\nAccuracité sur l'ensemble de test : {test_accuracy:.4f}")
print(classification_report(y_test, y_pred))

without parameter

In [ ]:
model_SVC=SVC(random_state=42)
model_SVC.fit(X_train, y_train)
y_pred_SVC=model_SVC.predict(X_test)

print(classification_report(y_test, y_pred_SVC))

Impact of n_shots on the model's performance

In [ ]:
# Analyser l'impact de n_shots sur la performance du modèle
from sklearn.model_selection import cross_val_score

n_shots_values = [10, 50, 100, 500, 1000, 5000, 10000]
results = []

print("Analyse de l'impact de n_shots sur la performance du modèle (F1-Score)...\n")
print("="*80)

for n_shots in n_shots_values:
    # Générer dataset avec ce n_shots
    df_temp, X_temp, y_temp = build_purity_classification_dataset(
        n_shots=n_shots,
        n_states_total=5000,
        mixed_proportion=0.3
    )
    
    # Préparer les données
    X_temp_data = df_temp[['X_mean', 'Y_mean', 'Z_mean']]
    X_temp_scaled = StandardScaler().fit_transform(X_temp_data)
    
    # Évaluer avec le modèle optimal trouvé avec F1-Score
    model = SVC(**grid_search.best_params_)
    cv_scores = cross_val_score(model, X_temp_scaled, y_temp, cv=5, scoring='f1_weighted')
    
    results.append({
        'n_shots': n_shots,
        'mean_f1': cv_scores.mean(),
        'std_f1': cv_scores.std()
    })
    
    print(f"n_shots={n_shots:5d} | F1-Score: {cv_scores.mean():.4f} ± {cv_scores.std():.4f}")

print("="*80)

# Créer DataFrame
results_df = pd.DataFrame(results)

# Visualiser la convergence avec F1-Score
fig, ax = plt.subplots(figsize=(11, 6))

ax.errorbar(results_df['n_shots'], results_df['mean_f1'], 
            yerr=results_df['std_f1'], 
            fmt='o-', capsize=5, linewidth=2.5, markersize=10, 
            color='darkblue', ecolor='lightblue', label='F1-Score ± Écart-type')

ax.set_xlabel('n_shots (nombre de mesures)', fontsize=12, fontweight='bold')
ax.set_ylabel('F1-Score (5-fold CV)', fontsize=12, fontweight='bold')
ax.set_title('Impact de n_shots sur la performance du modèle (F1-Score pondéré)', fontsize=14, fontweight='bold')
ax.set_xscale('log')
ax.grid(True, alpha=0.3)
ax.legend(fontsize=11, loc='lower right')
ax.set_ylim([0, 1.05])

plt.tight_layout()
plt.show()

# Recommandation
optimal_idx = results_df['mean_f1'].idxmax()
optimal_n_shots = results_df.loc[optimal_idx, 'n_shots']
optimal_f1 = results_df.loc[optimal_idx, 'mean_f1']
optimal_std = results_df.loc[optimal_idx, 'std_f1']

print(f"\n{'='*80}")
print("RÉSUMÉ - IMPACT DE N_SHOTS SUR LA PERFORMANCE (F1-SCORE)")
print(f"{'='*80}")
print(f"\n✓ n_shots optimal : {int(optimal_n_shots)}")
print(f"  F1-Score: {optimal_f1:.4f} ± {optimal_std:.4f}")
print(f"\n✓ Progression du F1-Score:")
print(f"  • n_shots minimal (10):     F1 = {results_df.loc[0, 'mean_f1']:.4f}")
print(f"  • n_shots optimal ({int(optimal_n_shots)}):  F1 = {optimal_f1:.4f} ⭐")
print(f"  • n_shots maximal (10000):  F1 = {results_df.loc[len(results_df)-1, 'mean_f1']:.4f}")
print(f"\n✓ Amélioration : {(optimal_f1 - results_df.loc[0, 'mean_f1'])*100:.2f}% par rapport à n_shots=10")
print(f"{'='*80}")

Convergence des (X_mean, Y_mean, Z_mean) selon le nombre de shots en 3D et 2D (avec loi des grands nombres)

In [ ]:
# Projection 3D des mesures (X_mean, Y_mean, Z_mean) selon n_shots
from mpl_toolkits.mplot3d import Axes3D

n_shots_test = [10, 100, 1000, 5000]
fig = plt.figure(figsize=(16, 12))

print("Génération des projections 3D pour différentes valeurs de n_shots...\n")

for idx, n_shots in enumerate(n_shots_test, 1):
    # Générer dataset
    df_temp, X_temp, y_temp = build_purity_classification_dataset(
        n_shots=n_shots,
        n_states_total=1000,
        mixed_proportion=0.3
    )
    
    # Sous-graphique 3D
    ax = fig.add_subplot(2, 2, idx, projection='3d')
    
    # Séparer les états purs et mixtes
    pure_mask = y_temp == 1
    mixed_mask = y_temp == 0
    
    # Scatter plot des états purs et mixtes
    ax.scatter(df_temp.loc[pure_mask, 'X_mean'],
               df_temp.loc[pure_mask, 'Y_mean'],
               df_temp.loc[pure_mask, 'Z_mean'],
               c='blue', marker='o', s=30, alpha=0.6, label='États purs')
    
    ax.scatter(df_temp.loc[mixed_mask, 'X_mean'],
               df_temp.loc[mixed_mask, 'Y_mean'],
               df_temp.loc[mixed_mask, 'Z_mean'],
               c='red', marker='x', s=50, alpha=0.6, label='États mixtes')
    
    # Ajouter la sphère de Bloch (rayon = 1)
    u = np.linspace(0, 2 * np.pi, 30)
    v = np.linspace(0, np.pi, 20)
    x_sphere = np.outer(np.cos(u), np.sin(v))
    y_sphere = np.outer(np.sin(u), np.sin(v))
    z_sphere = np.outer(np.ones(np.size(u)), np.cos(v))
    ax.plot_surface(x_sphere, y_sphere, z_sphere, alpha=0.1, color='gray')
    
    # Calcul des statistiques
    mean_radius = np.sqrt(df_temp['X_mean']**2 + df_temp['Y_mean']**2 + df_temp['Z_mean']**2).mean()
    std_radius = np.sqrt(df_temp['X_mean']**2 + df_temp['Y_mean']**2 + df_temp['Z_mean']**2).std()
    
    # Labels et titre
    ax.set_xlabel('X_mean', fontsize=10, fontweight='bold')
    ax.set_ylabel('Y_mean', fontsize=10, fontweight='bold')
    ax.set_zlabel('Z_mean', fontsize=10, fontweight='bold')
    ax.set_title(f'n_shots = {n_shots}\n(Rayon moyen: {mean_radius:.3f} ± {std_radius:.3f})', 
                 fontsize=11, fontweight='bold')
    ax.legend(fontsize=9)
    
    # Fixer les limites
    ax.set_xlim([-1.2, 1.2])
    ax.set_ylim([-1.2, 1.2])
    ax.set_zlim([-1.2, 1.2])
    
    print(f"n_shots={n_shots:5d} | Rayon moyen: {mean_radius:.4f} ± {std_radius:.4f}")

plt.suptitle('Projection 3D des mesures (X_mean, Y_mean, Z_mean) selon n_shots', 
             fontsize=14, fontweight='bold', y=0.995)
plt.tight_layout()
plt.show()

print("\n" + "="*80)
print("INTERPRÉTATION:")
print("="*80)
print("• Sphère grise: Sphère de Bloch idéale (rayon = 1)")
print("• Points bleus (o): États purs (devraient être près de la sphère)")
print("• Points rouges (x): États mixtes (devraient être à l'intérieur de la sphère)")
print("• Plus n_shots augmente, plus les mesures se rapprochent de la vraie géométrie")
print("• Avec n_shots faible: plus de bruit, points très dispersés")
print("• Avec n_shots élevé: mesures plus précises, distribution plus concentrée\n")

In [ ]:
# affichage des mesures (X_real, Y_real, Z_real) sur la sphère de Bloch 
        

LFGN to choose n_shots

In [ ]:
# Convergence de la distance entre état mesuré et état réel selon n_shots
# Application de la Loi des Grands Nombres

n_shots_values = [10, 25, 50, 100, 250, 500, 1000, 2500, 5000]
distance_stats = []

print("="*80)
print("ANALYSE DE CONVERGENCE: Distance mesure-réalité vs n_shots")
print("="*80)
print("\nConcept: La Loi des Grands Nombres prédit que l'erreur d'estimation")
print("décroît comme 1/√n_shots pour les moyennes statistiques.\n")

for n_shots in n_shots_values:
    # Générer dataset
    df_temp, X_temp, y_temp = build_purity_classification_dataset(
        n_shots=n_shots,
        n_states_total=5000,
        mixed_proportion=0.5
    )
    
    # Calculer la distance Euclidienne entre (X_mean, Y_mean, Z_mean) et (X_real, Y_real, Z_real)
    distances = np.sqrt(
        (df_temp['X_mean'] - df_temp['X_real'])**2 +
        (df_temp['Y_mean'] - df_temp['Y_real'])**2 +
        (df_temp['Z_mean'] - df_temp['Z_real'])**2
    )
    
    # Statistiques
    mean_distance = distances.mean()
    std_distance = distances.std()
    
    distance_stats.append({
        'n_shots': n_shots,
        'mean_distance': mean_distance,
        'std_distance': std_distance
    })
    
    print(f"n_shots={n_shots:5d} | Distance: {mean_distance:.6f} ± {std_distance:.6f}")

# Créer DataFrame
distance_df = pd.DataFrame(distance_stats)

# Visualisation: 2 graphiques
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Graphique 1: Distance moyenne avec erreur
ax1 = axes[0]
ax1.errorbar(distance_df['n_shots'], distance_df['mean_distance'],
             yerr=distance_df['std_distance'],
             fmt='o-', capsize=5, linewidth=2.5, markersize=8,
             color='darkblue', ecolor='lightblue', label='Moyenne ± Écart-type')
ax1.axhline(y=distance_df['mean_distance'].iloc[-1], color='green', linestyle='--',
            linewidth=2, label=f'Distance convergée (n_shots=5000)', alpha=0.7)
ax1.set_xlabel('n_shots', fontsize=12, fontweight='bold')
ax1.set_ylabel('Distance Euclidienne', fontsize=12, fontweight='bold')
ax1.set_title('Distance Mesure-Réalité vs n_shots', fontsize=13, fontweight='bold')
ax1.set_xscale('log')
ax1.grid(True, alpha=0.3)
ax1.legend(fontsize=10)

plt.tight_layout()
plt.show()

# Trouver le point d'inflexion (diminution < 10%)
distances = distance_df['mean_distance'].values
initial_distance = distances[0]

for i in range(1, len(distances)):
    reduction = (initial_distance - distances[i]) / initial_distance * 100
    if i > 1:
        prev_reduction = (initial_distance - distances[i-1]) / initial_distance * 100
        improvement = reduction - prev_reduction
        if improvement < 5:  # Amélioration inférieure à 5%
            optimal_idx = i - 1
            break
else:
    optimal_idx = len(distance_df) - 1

optimal_n_shots = distance_df.loc[optimal_idx, 'n_shots']
optimal_distance = distance_df.loc[optimal_idx, 'mean_distance']
optimal_reduction = (initial_distance - optimal_distance) / initial_distance * 100

optimal n_shots = 5000

In [ ]:
# Dataset creation

df_purity_5000, X, y = build_purity_classification_dataset(
    n_shots=5000,
    n_states_total=5000,
    mixed_proportion=0.5  # proportion of mixed states
)

print(f"Dataset shape: {df_purity_5000.shape}")
print(f"Labels distribution:\n{y.value_counts()}")
print(f"\nDataset columns:\n{df_purity_5000.columns.tolist()}")
print(f"\X columns (à revoir):\n{X.columns.tolist()}")

In [ ]:
df_purity_5000.describe()

In [ ]:
df_purity_5000.hist()

In [ ]:
X_purity_5000=df_purity_5000[['X_mean','Y_mean','Z_mean']]
y_purity_5000=df_purity_5000['label_purity']

# standardization
X_purity_5000_scaled=scaler.fit_transform(X_purity_5000)

#splitting the data
X_train, X_test, y_train, y_test = train_test_split(
    X_purity_5000_scaled, y_purity_5000, test_size=0.2, random_state=42, stratify=y_purity_5000)

# selecting the best parameters from the grid search
model_5000 = SVC(**grid_search.best_params_)
model_5000.fit(X_train, y_train)
y_pred = model_5000.predict(X_test)  

print(classification_report(y_test, y_pred))

In [ ]:
model_poly_2=SVC(C= 10, gamma= 'scale', kernel= 'poly', degree=2)
model_poly_2.fit(X_train, y_train)
y_pred_poly_2=model_poly_2.predict(X_test)
print(classification_report(y_test, y_pred_poly_2))

Plotting decision boundaries for the best models : Gaussian and degree 2 poly kernels

In [ ]:
# Visualisation des decision boundaries en 2D: RBF vs Polynomial (plan X_mean, Z_mean)

y_train_labels = y_train.values
y_test_labels = y_test.values

# Créer une grille 2D fine sur le plan (X, Z) - axes 0 et 2
h = 0.05  # pas de la grille pour 2D (plus fin que 3D)
x_min, x_max = X_train[:, 0].min() - 0.5, X_train[:, 0].max() + 0.5
z_min, z_max = X_train[:, 2].min() - 0.5, X_train[:, 2].max() + 0.5

xx_2d, zz_2d = np.meshgrid(np.arange(x_min, x_max, h),
                            np.arange(z_min, z_max, h))

# Pour les prédictions, il faut passer 3 features (X, Y, Z)
# On va utiliser la moyenne Y (0) pour les points de grille
y_mean_2d = np.zeros_like(xx_2d)

# Créer les points de grille avec Y = 0 (plan central)
# Format: [X, Y, Z]
grid_points = np.c_[xx_2d.ravel(), y_mean_2d.ravel(), zz_2d.ravel()]

# Prédictions sur la grille pour les deux modèles
Z_rbf_2d = model_5000.predict(grid_points)
Z_poly_2d = model_poly_2.predict(grid_points)

# Reshape pour contourf
Z_rbf_2d = Z_rbf_2d.reshape(xx_2d.shape)
Z_poly_2d = Z_poly_2d.reshape(xx_2d.shape)

# Créer la figure avec deux subplots 2D
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Subplot 1: RBF Kernel
ax1 = axes[0]

# Afficher la surface de décision RBF avec contourf
contour_rbf = ax1.contourf(xx_2d, zz_2d, Z_rbf_2d, levels=10, cmap='RdYlBu_r', alpha=0.6)
ax1.contour(xx_2d, zz_2d, Z_rbf_2d, levels=[0.5], colors='black', linewidths=2, label='Decision Boundary')

# Points d'entraînement (X et Z)
ax1.scatter(X_train[y_train_labels == 1, 0], X_train[y_train_labels == 1, 2],
           c='blue', marker='o', s=50, alpha=0.7, edgecolors='darkblue', linewidth=1, label='Train classe 1')
ax1.scatter(X_train[y_train_labels == 0, 0], X_train[y_train_labels == 0, 2],
           c='red', marker='x', s=100, alpha=0.7, linewidth=2, label='Train classe 0')

# Points de test (X et Z)
ax1.scatter(X_test[y_test_labels == 1, 0], X_test[y_test_labels == 1, 2],
           c='darkblue', marker='^', s=80, alpha=0.9, edgecolors='black', linewidth=1, label='Test classe 1')
ax1.scatter(X_test[y_test_labels == 0, 0], X_test[y_test_labels == 0, 2],
           c='darkred', marker='s', s=80, alpha=0.9, edgecolors='black', linewidth=1, label='Test classe 0')

ax1.set_xlim([x_min, x_max])
ax1.set_ylim([z_min, z_max])
ax1.set_xlabel('X_mean (standardisé)', fontsize=12, fontweight='bold')
ax1.set_ylabel('Z_mean (standardisé)', fontsize=12, fontweight='bold')
ax1.set_title('Decision Boundary - RBF Kernel (GridSearch)', fontsize=13, fontweight='bold')
ax1.legend(fontsize=10, loc='best')
ax1.grid(True, alpha=0.3)
plt.colorbar(contour_rbf, ax=ax1, label='Probabilité classe 1')

# Subplot 2: Polynomial Kernel (degree=2)
ax2 = axes[1]

# Afficher la surface de décision Polynomial avec contourf
contour_poly = ax2.contourf(xx_2d, zz_2d, Z_poly_2d, levels=10, cmap='RdYlBu_r', alpha=0.6)
ax2.contour(xx_2d, zz_2d, Z_poly_2d, levels=[0.5], colors='black', linewidths=2, label='Decision Boundary')

# Points d'entraînement (X et Z)
ax2.scatter(X_train[y_train_labels == 1, 0], X_train[y_train_labels == 1, 2],
           c='blue', marker='o', s=50, alpha=0.7, edgecolors='darkblue', linewidth=1, label='Train classe 1')
ax2.scatter(X_train[y_train_labels == 0, 0], X_train[y_train_labels == 0, 2],
           c='red', marker='x', s=100, alpha=0.7, linewidth=2, label='Train classe 0')

# Points de test (X et Z)
ax2.scatter(X_test[y_test_labels == 1, 0], X_test[y_test_labels == 1, 2],
           c='darkblue', marker='^', s=80, alpha=0.9, edgecolors='black', linewidth=1, label='Test classe 1')
ax2.scatter(X_test[y_test_labels == 0, 0], X_test[y_test_labels == 0, 2],
           c='darkred', marker='s', s=80, alpha=0.9, edgecolors='black', linewidth=1, label='Test classe 0')

ax2.set_xlim([x_min, x_max])
ax2.set_ylim([z_min, z_max])
ax2.set_xlabel('X_mean (standardisé)', fontsize=12, fontweight='bold')
ax2.set_ylabel('Z_mean (standardisé)', fontsize=12, fontweight='bold')
ax2.set_title('Decision Boundary - Polynomial (degree=2)', fontsize=13, fontweight='bold')
ax2.legend(fontsize=10, loc='best')
ax2.grid(True, alpha=0.3)
plt.colorbar(contour_poly, ax=ax2, label='Probabilité classe 1')

plt.suptitle('Comparaison des Decision Boundaries: RBF vs Polynomial (Plan X_mean, Z_mean)', 
             fontsize=15, fontweight='bold', y=0.98)
plt.tight_layout()
plt.show()

# Afficher les statistiques de comparaison
print("="*80)
print("COMPARAISON DES KERNELS - DECISION BOUNDARIES EN 2D (X, Z)")
print("="*80)

# Prédictions pour évaluation sur le plan 2D avec Y=0
from sklearn.metrics import f1_score, classification_report

y_pred_rbf_2d = model_5000.predict(X_test)
y_pred_poly_2d = model_poly_2.predict(X_test)

# Utiliser le F1-score pondéré
rbf_f1_score_2d = f1_score(y_test_labels, y_pred_rbf_2d, average='weighted')
poly_f1_score_2d = f1_score(y_test_labels, y_pred_poly_2d, average='weighted')

print(f"\n✓ RBF Kernel (GridSearch):")
print(f"  Paramètres: {grid_search.best_params_}")
print(f"  F1-Score (weighted) test (3D complet): {rbf_f1_score_2d:.4f}")

print(f"\n✓ Polynomial Kernel (degree=2):")
print(f"  Paramètres: C=10, gamma='scale', kernel='poly', degree=2")
print(f"  F1-Score (weighted) test (3D complet): {poly_f1_score_2d:.4f}")

print(f"\n✓ Différence de F1-Score (weighted): {abs(rbf_f1_score_2d - poly_f1_score_2d):.4f}")
print(f"  → RBF est meilleur: {rbf_f1_score_2d > poly_f1_score_2d}")

print("\n✓ Visualisation 2D (Plan X_mean, Z_mean avec Y=0):")
print(f"  • Fond coloré: Région de décision (bleu=classe 1, rouge=classe 0)")
print(f"  • Ligne noire: Frontière de décision (boundary)")
print(f"  • Cercles bleus (o): Points d'entraînement classe 1")
print(f"  • Croix rouges (x): Points d'entraînement classe 0")
print(f"  • Triangles bleus (^): Points de test classe 1")
print(f"  • Carrés rouges (s): Points de test classe 0")
print(f"  • Plan XZ avec Y=0 (coupe centrale de la sphère de Bloch)")
print(f"  • Le RBF crée des régions courbes (non-linéaires)")
print(f"  • Le Polynomial crée des régions facettées (quadratiques)")
print("="*80)


GridSearch: Optimisation conjointe de n_shots et des paramètres SVC pour le meilleur F1-Score

For balanced dataset : mixed_proportion=0.5

In [ ]:
# GridSearch pour optimiser n_shots ET paramètres SVC
from sklearn.model_selection import ParameterGrid
import time

# Paramètres à optimiser
n_shots_values = [10, 50, 100, 500, 1000, 5000]
svc_params = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],  # Pas de 'poly' par défaut
    'gamma': ['scale', 'auto'],
    'degree': [2]  # Paramètre de degré pour polynomial
}

# Créer la grille des paramètres
svc_param_grid = list(ParameterGrid(svc_params))

# Ajouter manuellement le kernel polynomial de degré 2
svc_param_grid_extended = []
for params in svc_param_grid:
    svc_param_grid_extended.append(params)

# Ajouter les combinaisons avec polynomial de degré 2
for C in [0.1, 1, 10]:
    for gamma in ['scale', 'auto']:
        svc_param_grid_extended.append({
            'C': C,
            'kernel': 'poly',
            'gamma': gamma,
            'degree': 2
        })

svc_param_grid = svc_param_grid_extended

# Résultats
results_grid = []
total_combinations = len(n_shots_values) * len(svc_param_grid)

print("="*100)
print("GRIDSEARCH: OPTIMISATION CONJOINTE DE n_shots ET PARAMÈTRES SVC")
print("="*100)
print(f"\nParamètres SVC: {len(svc_param_grid)} combinaisons")
print(f"Valeurs n_shots: {len(n_shots_values)} valeurs")
print(f"Total d'itérations: {total_combinations}\n")
print(f"Configuration:")
print(f"  • n_states_total = 5000")
print(f"  • mixed_proportion = 0.5")
print(f"  • Kernels: linear, rbf, poly(degree=2)")
print(f"  • Métrique: F1-Score (weighted)")
print(f"  • Cross-validation: 5-fold")
print("="*100 + "\n")

iteration = 0
start_time = time.time()

for n_shots in n_shots_values:
    print(f"\n[n_shots = {n_shots:5d}] Génération dataset et GridSearch SVC...")
    print("-" * 100)
    
    # Générer le dataset avec ce n_shots
    df_temp, _, y_temp = build_purity_classification_dataset(
        n_shots=n_shots,
        n_states_total=5000,
        mixed_proportion=0.5
    )
    
    X_temp_data = df_temp[['X_mean', 'Y_mean', 'Z_mean']]
    X_temp_scaled = StandardScaler().fit_transform(X_temp_data)
    
    # Tester chaque combinaison de paramètres SVC
    best_f1_for_n_shots = 0
    best_config_for_n_shots = None
    
    for params in svc_param_grid:
        iteration += 1
        
        # Créer et évaluer le modèle
        svc_model = SVC(**params)
        
        from sklearn.model_selection import cross_val_score
        cv_scores = cross_val_score(
            svc_model, X_temp_scaled, y_temp, 
            cv=5, scoring='f1_weighted'
        )
        
        # Enregistrer les résultats
        result = {
            'n_shots': n_shots,
            'C': params['C'],
            'kernel': params['kernel'],
            'gamma': params['gamma'],
            'degree': params.get('degree', None),
            'mean_f1': cv_scores.mean(),
            'std_f1': cv_scores.std()
        }
        results_grid.append(result)
        
        # Afficher chaque exécution
        degree_str = f", degree={params.get('degree', 'N/A')}" if params['kernel'] == 'poly' else ""
        print(f"  n_shots={n_shots:5d} | C={params['C']:4.1f} | kernel={params['kernel']:6s} | gamma={params['gamma']:6s}{degree_str} | F1={cv_scores.mean():.4f} ± {cv_scores.std():.4f}")
        
        # Tracker le meilleur pour ce n_shots
        if cv_scores.mean() > best_f1_for_n_shots:
            best_f1_for_n_shots = cv_scores.mean()
            best_config_for_n_shots = result
    
    # Afficher le meilleur pour ce n_shots
    print(f"  → MEILLEUR POUR n_shots={n_shots}: F1={best_f1_for_n_shots:.4f} (C={best_config_for_n_shots['C']}, kernel={best_config_for_n_shots['kernel']}, gamma={best_config_for_n_shots['gamma']})")

# Créer un DataFrame avec tous les résultats
results_grid_df = pd.DataFrame(results_grid)

# Trouver le meilleur résultat global
best_idx = results_grid_df['mean_f1'].idxmax()
best_result = results_grid_df.loc[best_idx]

elapsed_time = time.time() - start_time

print("\n" + "="*100)
print("RÉSULTATS DU GRIDSEARCH - MEILLEURE CONFIGURATION GLOBALE")
print("="*100)
print(f"\n✓ MEILLEUR n_shots: {int(best_result['n_shots'])}")
print(f"\n✓ MEILLEURS PARAMÈTRES SVC:")
print(f"  • C: {best_result['C']}")
print(f"  • kernel: {best_result['kernel']}")
print(f"  • gamma: {best_result['gamma']}")
if best_result['degree'] is not None and best_result['kernel'] == 'poly':
    print(f"  • degree: {int(best_result['degree'])}")
print(f"\n✓ F1-Score (weighted): {best_result['mean_f1']:.4f} ± {best_result['std_f1']:.4f}")
print(f"\n✓ Temps d'exécution total: {elapsed_time:.2f} secondes")
print("="*100)


for mixed proportion = 0.1

In [ ]:
from sklearn.model_selection import ParameterGrid
import time

# Paramètres à optimiser
n_shots_values = [10, 50, 100, 500, 1000, 5000]
svc_params = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],  # Pas de 'poly' par défaut
    'gamma': ['scale', 'auto'],
    'degree': [2]  # Paramètre de degré pour polynomial
}

# Créer la grille des paramètres
svc_param_grid = list(ParameterGrid(svc_params))

# Ajouter manuellement le kernel polynomial de degré 2
svc_param_grid_extended = []
for params in svc_param_grid:
    svc_param_grid_extended.append(params)

# Ajouter les combinaisons avec polynomial de degré 2
for C in [0.1, 1, 10]:
    for gamma in ['scale', 'auto']:
        svc_param_grid_extended.append({
            'C': C,
            'kernel': 'poly',
            'gamma': gamma,
            'degree': 2
        })

svc_param_grid = svc_param_grid_extended

# Résultats
results_grid = []
total_combinations = len(n_shots_values) * len(svc_param_grid)

print("="*100)
print("GRIDSEARCH: OPTIMISATION CONJOINTE DE n_shots ET PARAMÈTRES SVC")
print("="*100)
print(f"\nParamètres SVC: {len(svc_param_grid)} combinaisons")
print(f"Valeurs n_shots: {len(n_shots_values)} valeurs")
print(f"Total d'itérations: {total_combinations}\n")
print(f"Configuration:")
print(f"  • n_states_total = 5000")
print(f"  • mixed_proportion = 0.1")
print(f"  • Kernels: linear, rbf, poly(degree=2)")
print(f"  • Métrique: F1-Score (weighted)")
print(f"  • Cross-validation: 5-fold")
print("="*100 + "\n")

iteration = 0
start_time = time.time()

for n_shots in n_shots_values:
    print(f"\n[n_shots = {n_shots:5d}] Génération dataset et GridSearch SVC...")
    print("-" * 100)
    
    # Générer le dataset avec ce n_shots
    df_temp, _, y_temp = build_purity_classification_dataset(
        n_shots=n_shots,
        n_states_total=5000,
        mixed_proportion=0.1
    )
    
    X_temp_data = df_temp[['X_mean', 'Y_mean', 'Z_mean']]
    X_temp_scaled = StandardScaler().fit_transform(X_temp_data)
    
    # Tester chaque combinaison de paramètres SVC
    best_f1_for_n_shots = 0
    best_config_for_n_shots = None
    
    for params in svc_param_grid:
        iteration += 1
        
        # Créer et évaluer le modèle
        svc_model = SVC(**params)
        
        from sklearn.model_selection import cross_val_score
        cv_scores = cross_val_score(
            svc_model, X_temp_scaled, y_temp, 
            cv=5, scoring='f1_weighted'
        )
        
        # Enregistrer les résultats
        result = {
            'n_shots': n_shots,
            'C': params['C'],
            'kernel': params['kernel'],
            'gamma': params['gamma'],
            'degree': params.get('degree', None),
            'mean_f1': cv_scores.mean(),
            'std_f1': cv_scores.std()
        }
        results_grid.append(result)
        
        # Afficher chaque exécution
        degree_str = f", degree={params.get('degree', 'N/A')}" if params['kernel'] == 'poly' else ""
        print(f"  n_shots={n_shots:5d} | C={params['C']:4.1f} | kernel={params['kernel']:6s} | gamma={params['gamma']:6s}{degree_str} | F1={cv_scores.mean():.4f} ± {cv_scores.std():.4f}")
        
        # Tracker le meilleur pour ce n_shots
        if cv_scores.mean() > best_f1_for_n_shots:
            best_f1_for_n_shots = cv_scores.mean()
            best_config_for_n_shots = result
    
    # Afficher le meilleur pour ce n_shots
    print(f"  → MEILLEUR POUR n_shots={n_shots}: F1={best_f1_for_n_shots:.4f} (C={best_config_for_n_shots['C']}, kernel={best_config_for_n_shots['kernel']}, gamma={best_config_for_n_shots['gamma']})")

# Créer un DataFrame avec tous les résultats
results_grid_df = pd.DataFrame(results_grid)

# Trouver le meilleur résultat global
best_idx = results_grid_df['mean_f1'].idxmax()
best_result = results_grid_df.loc[best_idx]

elapsed_time = time.time() - start_time

print("\n" + "="*100)
print("RÉSULTATS DU GRIDSEARCH - MEILLEURE CONFIGURATION GLOBALE")
print("="*100)
print(f"\n✓ MEILLEUR n_shots: {int(best_result['n_shots'])}")
print(f"\n✓ MEILLEURS PARAMÈTRES SVC:")
print(f"  • C: {best_result['C']}")
print(f"  • kernel: {best_result['kernel']}")
print(f"  • gamma: {best_result['gamma']}")
if best_result['degree'] is not None and best_result['kernel'] == 'poly':
    print(f"  • degree: {int(best_result['degree'])}")
print(f"\n✓ F1-Score (weighted): {best_result['mean_f1']:.4f} ± {best_result['std_f1']:.4f}")
print(f"\n✓ Temps d'exécution total: {elapsed_time:.2f} secondes")
print("="*100)


In [ ]:
from sklearn.model_selection import ParameterGrid
import time

# Paramètres à optimiser
n_shots_values = [10, 50, 100, 500, 1000, 5000]
svc_params = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],  # Pas de 'poly' par défaut
    'gamma': ['scale', 'auto'],
    'degree': [2]  # Paramètre de degré pour polynomial
}

# Créer la grille des paramètres
svc_param_grid = list(ParameterGrid(svc_params))

# Ajouter manuellement le kernel polynomial de degré 2
svc_param_grid_extended = []
for params in svc_param_grid:
    svc_param_grid_extended.append(params)

# Ajouter les combinaisons avec polynomial de degré 2
for C in [0.1, 1, 10]:
    for gamma in ['scale', 'auto']:
        svc_param_grid_extended.append({
            'C': C,
            'kernel': 'poly',
            'gamma': gamma,
            'degree': 2
        })

svc_param_grid = svc_param_grid_extended

# Résultats
results_grid = []
total_combinations = len(n_shots_values) * len(svc_param_grid)

print("="*100)
print("GRIDSEARCH: OPTIMISATION CONJOINTE DE n_shots ET PARAMÈTRES SVC")
print("="*100)
print(f"\nParamètres SVC: {len(svc_param_grid)} combinaisons")
print(f"Valeurs n_shots: {len(n_shots_values)} valeurs")
print(f"Total d'itérations: {total_combinations}\n")
print(f"Configuration:")
print(f"  • n_states_total = 5000")
print(f"  • mixed_proportion = 0.3")
print(f"  • Kernels: linear, rbf, poly(degree=2)")
print(f"  • Métrique: F1-Score (weighted)")
print(f"  • Cross-validation: 5-fold")
print("="*100 + "\n")

iteration = 0
start_time = time.time()

for n_shots in n_shots_values:
    print(f"\n[n_shots = {n_shots:5d}] Génération dataset et GridSearch SVC...")
    print("-" * 100)
    
    # Générer le dataset avec ce n_shots
    df_temp, _, y_temp = build_purity_classification_dataset(
        n_shots=n_shots,
        n_states_total=5000,
        mixed_proportion=0.3
    )
    
    X_temp_data = df_temp[['X_mean', 'Y_mean', 'Z_mean']]
    X_temp_scaled = StandardScaler().fit_transform(X_temp_data)
    
    # Tester chaque combinaison de paramètres SVC
    best_f1_for_n_shots = 0
    best_config_for_n_shots = None
    
    for params in svc_param_grid:
        iteration += 1
        
        # Créer et évaluer le modèle
        svc_model = SVC(**params)
        
        from sklearn.model_selection import cross_val_score
        cv_scores = cross_val_score(
            svc_model, X_temp_scaled, y_temp, 
            cv=5, scoring='f1_weighted'
        )
        
        # Enregistrer les résultats
        result = {
            'n_shots': n_shots,
            'C': params['C'],
            'kernel': params['kernel'],
            'gamma': params['gamma'],
            'degree': params.get('degree', None),
            'mean_f1': cv_scores.mean(),
            'std_f1': cv_scores.std()
        }
        results_grid.append(result)
        
        # Afficher chaque exécution
        degree_str = f", degree={params.get('degree', 'N/A')}" if params['kernel'] == 'poly' else ""
        print(f"  n_shots={n_shots:5d} | C={params['C']:4.1f} | kernel={params['kernel']:6s} | gamma={params['gamma']:6s}{degree_str} | F1={cv_scores.mean():.4f} ± {cv_scores.std():.4f}")
        
        # Tracker le meilleur pour ce n_shots
        if cv_scores.mean() > best_f1_for_n_shots:
            best_f1_for_n_shots = cv_scores.mean()
            best_config_for_n_shots = result
    
    # Afficher le meilleur pour ce n_shots
    print(f"  → MEILLEUR POUR n_shots={n_shots}: F1={best_f1_for_n_shots:.4f} (C={best_config_for_n_shots['C']}, kernel={best_config_for_n_shots['kernel']}, gamma={best_config_for_n_shots['gamma']})")

# Créer un DataFrame avec tous les résultats
results_grid_df = pd.DataFrame(results_grid)

# Trouver le meilleur résultat global
best_idx = results_grid_df['mean_f1'].idxmax()
best_result = results_grid_df.loc[best_idx]

elapsed_time = time.time() - start_time

print("\n" + "="*100)
print("RÉSULTATS DU GRIDSEARCH - MEILLEURE CONFIGURATION GLOBALE")
print("="*100)
print(f"\n✓ MEILLEUR n_shots: {int(best_result['n_shots'])}")
print(f"\n✓ MEILLEURS PARAMÈTRES SVC:")
print(f"  • C: {best_result['C']}")
print(f"  • kernel: {best_result['kernel']}")
print(f"  • gamma: {best_result['gamma']}")
if best_result['degree'] is not None and best_result['kernel'] == 'poly':
    print(f"  • degree: {int(best_result['degree'])}")
print(f"\n✓ F1-Score (weighted): {best_result['mean_f1']:.4f} ± {best_result['std_f1']:.4f}")
print(f"\n✓ Temps d'exécution total: {elapsed_time:.2f} secondes")
print("="*100)
